In [1]:
from upsetplot import plot
from matplotlib import pyplot
import pandas as pd
import csv, copy

In [3]:
file=open('/Users/zacharymcgrath/Downloads/HpHtest.csv')
file_contents=csv.reader(file, delimiter=",")

hits = []
row_number = 1
for row in file_contents:
    peptides = list(row)
    if row_number == 1:
        number_of_sets = len(peptides)
        for i in range(0, number_of_sets):
            hits.append([])
        set_names = peptides
    else:
        for i in range(0, len(peptides)):
            if (peptides[i] not in hits[i]) and (peptides[i] != ""):
                hits[i].append(peptides[i])  
    row_number = row_number + 1
    
intersections = [["TRUE"],["FALSE"]]
for i in range(1, number_of_sets):
    duplicate = copy.deepcopy(intersections)
    for j in range(0, len(intersections)):
        intersections[j].append("TRUE")
        duplicate[j].append("FALSE")
    intersections = intersections + duplicate
    
for i in range(0, len(intersections)):
    if "TRUE" not in intersections[i]:
        del(intersections[i])
        
all_hits = []
for i in range(0, len(hits)):
    for j in range(0, len(hits[i])):
        if hits[i][j] not in all_hits:
            all_hits.append(hits[i][j])
    
def overlap_finder(all_hits, hits, intersections):
    found = []
    for i in range(len(all_hits)):
        found.append([])
        for j in range(0, len(hits)):
            if all_hits[i] in hits[j]:
                hits[j].remove(all_hits[i])
                found[i].append("TRUE")
            else:
                found[i].append("FALSE")
    
    for i in range(0, len(intersections)):
        criterion = intersections[i]
        cardinality = 0
        intersection_peptides = []
        for j in range(0, len(found)):
            if found[j] == criterion:
                cardinality = cardinality + 1
                intersection_peptides.append(all_hits[j])
        intersections[i].append(cardinality)
        intersections[i].append(intersection_peptides)
    
    return(intersections)
        
intersections = overlap_finder(all_hits, hits, intersections)

# for i in range(0, len(intersections)):
#     print(intersections[i][:len(intersections[i])-1])
    
# print()
    
# remove the intersecting combos
file.close()
truth_table_counts = []
for intersection in intersections:
    truth_table_counts.append(intersection[:-1])
    
df = pd.DataFrame(truth_table_counts, columns=[x for x in range(6)] + ['count'])
df.head()
               


,0,1,2,3,4,5,count
0,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,18
1,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,3
2,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,4
3,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE,9
4,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,6


In [102]:
def intersection_evaluation(df: pd.DataFrame, overlaps=[]) -> pd.DataFrame:
    '''
    Evaluate the intersection of every column evaluation for the column pairs provide
    in overlaps. 
    
    Example:
        inputs:
            cat    dog    fish
            1      4      9
            19     19     7
            8      27     8
            
        overlaps:
            [['cat', 'dog'], ['cat', 'fish'], ['cat', 'dog', 'fish']]
            
        Outputs:
            cat    dog    fish
            True   True   False    1
            True   False  True     1
            True   True   True     0
            
    Inputs:
        df:          (pd.DataFrame) the input dataframe
        overlaps:    (list) the list of overlap pairs to check
    Outputs:
        (pd.DataFrame) the resulting output dataframe
    '''
    # turn a tuple into a hashable str
    comp_key = lambda x: '_'.join(x)
    
    # keep intersection count
    comps = defaultdict(lambda: 0)
    
    # listify all keys and create a lambda for the case where we want all but 1 key
    all_keys = list(df.columns)
    all_keys_but = lambda x: [k for k in all_keys if k != x]
    
    # function to use in the df apply
    def row_intersection(row):
        
        # go through each set of parameters to find the overlap of the set
        for comp_set in overlaps:
            
            # how much to increment for this intersection
            increment = 0
            
            # 1 key is the special case
            if len(comp_set) == 1:
                
                # increment is 0 if any of the other columns have the same value as this one
                # otherwise it is unique
                increment = 0 if \
                    any([row[key] == row[comp_set[0]] for key in all_keys_but(comp_set[0])]) \
                    else 1
            
            # at least a pair is being compared
            else:
                
                # increment if all keys in the comparison set have the same value otherwise dont
                increment = 1 if \
                    all([row[key] == row[comp_set[0]] for key in comp_set[1:]]) \
                    else 0
                
            # increment by value
            comps[comp_key(comp_set)] += increment
            
    # apply intersection to all rows
    df.apply(row_intersection, axis=1)
    
    # turn comps into a dataframe
    all_rows = []
    
    for comp_set, count in comps.items():
        
        # unset the key
        keys = comp_set.split('_')
        row = [key in keys for key in all_keys]
        all_rows.append(row + [count])
        
    return pd.DataFrame(all_rows, columns=all_keys + ['intersection'])
        

In [93]:
hpdf = pd.read_csv('~/Downloads/HpHtest.csv')

all_combos = []
for i in range(2, len(hpdf.columns)):
    all_combos += combinations(hpdf.columns, i)
    
all_combos += [[x] for x in hpdf.columns]
all_combos = [list(x) for x in all_combos] + [list(hpdf.columns)]


result = intersection_evaluation(hpdf, all_combos)

In [94]:
result.tail(10)

,HpH1,HpH2,HpH3,HpH4,HpH5,HpH6,intersection
53,True,True,False,True,True,True,0
54,True,False,True,True,True,True,0
55,False,True,True,True,True,True,0
56,True,False,False,False,False,False,592
57,False,True,False,False,False,False,589
58,False,False,True,False,False,False,599
59,False,False,False,True,False,False,600
60,False,False,False,False,True,False,603
61,False,False,False,False,False,True,599
62,True,True,True,True,True,True,0


In [98]:
import pandas as pd
from itertools import combinations
from collections import defaultdict

def intersection_evaluation(df: pd.DataFrame, overlaps=[]) -> pd.DataFrame:
    '''
    Evaluate the intersection of every column evaluation for the column pairs provide
    in overlaps. 
    
    Example:
        inputs:
            cat    dog    fish
            1      4      9
            19     19     7
            8      27     8
            
        overlaps:
            [['cat', 'dog'], ['cat', 'fish'], ['cat', 'dog', 'fish']]
            
        Outputs:
            cat    dog    fish
            True   True   False    1
            True   False  True     1
            True   True   True     0
            
    Inputs:
        df:          (pd.DataFrame) the input dataframe
        overlaps:    (list) the list of overlap pairs to check
    Outputs:
        (pd.DataFrame) the resulting output dataframe
    '''
    # turn a tuple into a hashable str
    comp_key = lambda x: '_'.join(x)
    
    # keep intersection count
    comps = defaultdict(lambda: 0)
    
    # listify all keys and create a lambda for the case where we want all but 1 key
    all_keys = list(df.columns)
    all_keys_but = lambda x: [k for k in all_keys if k != x]
    
    # function to use in the df apply
    def row_intersection(row):
        
        # go through each set of parameters to find the overlap of the set
        for comp_set in overlaps:
            
            # how much to increment for this intersection
            increment = 0
            
            # 1 key is the special case
            if len(comp_set) == 1:
                
                # increment is 0 if any of the other columns have the same value as this one
                # otherwise it is unique
                increment = 0 if \
                    any([row[key] == row[comp_set[0]] for key in all_keys_but(comp_set[0])]) \
                    else 1
            
            # at least a pair is being compared
            else:
                
                # increment if all keys in the comparison set have the same value otherwise dont
                increment = 1 if \
                    all([row[key] == row[comp_set[0]] for key in comp_set[1:]]) \
                    else 0
                
            # increment by value
            comps[comp_key(comp_set)] += increment
            
    # apply intersection to all rows
    df.apply(row_intersection, axis=1)
    
    # turn comps into a dataframe
    all_rows = []
    
    for comp_set, count in comps.items():
        
        # unset the key
        keys = comp_set.split('_')
        row = [key in keys for key in all_keys]
        all_rows.append(row + [count])
        
    return pd.DataFrame(all_rows, columns=all_keys + ['intersection'])

# load the csv into pandas dataframe
hpdf = pd.read_csv('~/Downloads/HpHtest.csv')

# create all combinations of keys
all_combos = []
for i in range(1, len(hpdf.columns) + 1):
    all_combos += combinations(hpdf.columns, i)
all_combos = [list(x) for x in all_combos] 

# save the result in a dataframe
result = intersection_evaluation(hpdf, all_combos)
        

In [99]:
result.tail(10)

,HpH1,HpH2,HpH3,HpH4,HpH5,HpH6,intersection
53,False,True,True,False,True,True,0
54,False,True,False,True,True,True,0
55,False,False,True,True,True,True,0
56,True,True,True,True,True,False,0
57,True,True,True,True,False,True,0
58,True,True,True,False,True,True,0
59,True,True,False,True,True,True,0
60,True,False,True,True,True,True,0
61,False,True,True,True,True,True,0
62,True,True,True,True,True,True,0
